In [ ]:
from functools import cache
from itertools import permutations

import numpy as np
from tqdm.auto import tqdm

In [ ]:
def is_valid_route(idx, steps, keypad, offset_map):
    valid = True
    for step in steps:
        idx = idx + offset_map[step]
        if not keypad[*idx]:
            valid = False
    return valid

In [ ]:
def get_combinations(steps_options):
    currs = [""]
    nexts = []
    for step_options in steps_options:
        for curr in currs:
            for step_option in step_options:
                nexts.append(curr + step_option)
        currs = nexts
        nexts = []

    return currs

In [ ]:
def flatten_list(outer):
    return [ele for inner in outer for ele in inner]

In [ ]:
def only_shortest(full):
    min_len = min(len(x) for x in full)
    return [x for x in full if len(x) == min_len]

In [ ]:
def get_steps_options(code, keypad, offset_map):

    idx_curr = np.argwhere(keypad == "A")[0]
    steps_options = []

    for code_digit in code:

        idx_target = np.argwhere(keypad == code_digit)[0]
        dy, dx = idx_target - idx_curr

        steps_raw = ""
        if dy < 0:
            steps_raw += "^" * abs(dy)
        elif dy > 0:
            steps_raw += "v" * abs(dy)
        if dx < 0:
            steps_raw += "<" * abs(dx)
        elif dx > 0:
            steps_raw += ">" * abs(dx)

        steps_options.append(
            [
                x + "A"
                for x in set(["".join(x) for x in permutations(steps_raw)])
                if is_valid_route(idx_curr.copy(), x, keypad, offset_map)
            ]
        )

        idx_curr = idx_target

    steps_options = get_combinations(steps_options)

    return steps_options

In [ ]:
with open("data/day21/input.txt", "r") as file:
    numeric_codes_raw = file.read()

In [ ]:
numeric_codes = numeric_codes_raw.split("\n")

In [ ]:
numeric_keypad = np.array(
    [["7", "8", "9"], ["4", "5", "6"], ["1", "2", "3"], ["", "0", "A"]]
)
numeric_keypad

In [ ]:
directional_keypad = np.array([["", "^", "A"], ["<", "v", ">"]])
directional_keypad

In [ ]:
offset_map = {
    "^": (-1, 0),
    "v": (1, 0),
    "<": (0, -1),
    ">": (0, 1),
}

In [ ]:
@cache
def get_steps_options_rec(steps_options, target_depth, depth=0):
    if not isinstance(steps_options, list):
        steps_options = [steps_options]

    keypad = numeric_keypad if depth == 0 else directional_keypad
    int_steps_options = only_shortest(
        flatten_list(
            [get_steps_options(step, keypad, offset_map) for step in steps_options]
        )
    )

    if depth == target_depth:
        return int_steps_options
    else:
        return only_shortest(
            flatten_list(
                [
                    get_steps_options_rec(x, target_depth, depth + 1)
                    for x in int_steps_options
                ]
            )
        )

In [ ]:
test_res = {
    "029A": "<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A",
    "980A": "<v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A",
    "179A": "<v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A",
    "456A": "<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A",
    "379A": "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A",
}

## Part 1

In [ ]:
result = {
    numeric_code: get_steps_options_rec(numeric_code, 2)
    for numeric_code in tqdm(numeric_codes)
}

In [ ]:
score = 0
for numeric, direction in result.items():
    if numeric in test_res:
        assert test_res[numeric] in direction
    score += int(numeric[:-1]) * len(direction[0])
score

## Part 2